<a href="https://colab.research.google.com/github/yy20190404/db_covid/blob/master/COVID19Dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# COVID19-Dashboard

## Collect COVID19 data from web

###　備忘録

herokuはPaaS（Platform as a Service）のひとつで、無料でプログラムを実行しwebに公開することができる。サーバー構築をしなくても公開できる。

herokuはGithubのリポジトリを利用しており、利用にあたってGithubのアカウントが必要。GitとHerokuをローカルPCにインストールし、CLIベースで利用するのが基本。

herokuは'git push heroku master'でプッシュされた際のファイルから自動的にファイルに合ったbuildpackをインストールする。Pythonの場合、'requirements.txt'があるとpythonと認識し、python buildpackageを使用する。

Gitコマンド：
・　リモートリポジトリの確認方法
  git config --get remote.origin.url
  git remote -v
  git remote show -n origin
  git remote show origin
  cat .git/config
  git config -l
  ((git remote show --help))
・　リモートリポジトリ追加
　git remote add origin username@ipaddress:/home/username/test.git
・　忘れがちなコマンド
　https://qiita.com/m_aki/items/3e0fbc0cf75eca96128b

Nginx：
　Buildpacのインストール必要　heroku dashborad settingsからインストールできる。
　https://github.com/heroku/heroku-buildpack-nginx

Gunicorn：
　pipでインストール可能。
　Dash（Flask）のapp定義の直下に'server = app.server'が必要。
　! gunicorn app:serverで起動
　http://127.0.0.1:8000


### install python libraries, make directories


In [0]:
!python -m pip install --upgrade pip
!pip install requests
!python -m pip install requests-html
!python -m pip install seaborn
!pip install japanize-matplotlib
!python -m pip install flask
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
!mkdir templates
!mkdir static
!mkdir static/img
!mkdir static/csv
from google.colab import drive
drive.mount("/content/drive")
!cp drive/'My Drive'/'Colab Notebooks'/japan-map-master/japan-map.jquery.json templates/
!cp drive/'My Drive'/'Colab Notebooks'/japan-map-master/jquery.japan-map.js templates/
!cp drive/'My Drive'/'Colab Notebooks'/japan-map-master/jquery.japan-map.min.js templates/

### Reshape data to use dashboard

In [0]:
dldata = """

#!/usr/bin/env python
#-*- coding:utf-8 -*-

import datetime
import gc
import os
import sys

import numpy as np
import pandas as pd

import requests
from requests_html import HTMLSession

def get_converted_multi_columns(df, just_second=False, to_snake_case=True):
  if just_second:
    return [col[1] for col in df.columns.values]
  else:
    if to_snake_case:
        return [col[0] + '_' + col[1] for col in df.columns.values]
    else:
        return [col[0] + col[1].capitalize() for col in df.columns.values]

def down_load():
  HEADERS_DIC = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.97 Safari/537.36"}
  URLWW1 = "https://data.humdata.org/hxlproxy/api/data-preview.csv?url=https%3A%2F%2Fraw.githubusercontent.com%2FCSSEGISandData%2FCOVID-19%2Fmaster%2Fcsse_covid_19_data%2Fcsse_covid_19_time_series%2Ftime_series_covid19_confirmed_global.csv&filename=time_series_covid19_confirmed_global.csv"
  URLWW2 = "https://data.humdata.org/hxlproxy/api/data-preview.csv?url=https%3A%2F%2Fraw.githubusercontent.com%2FCSSEGISandData%2FCOVID-19%2Fmaster%2Fcsse_covid_19_data%2Fcsse_covid_19_time_series%2Ftime_series_covid19_deaths_global.csv&filename=time_series_covid19_deaths_global.csv"
  URLWW3 = "https://data.humdata.org/hxlproxy/api/data-preview.csv?url=https%3A%2F%2Fraw.githubusercontent.com%2FCSSEGISandData%2FCOVID-19%2Fmaster%2Fcsse_covid_19_data%2Fcsse_covid_19_time_series%2Ftime_series_covid19_recovered_global.csv&filename=time_series_covid19_recovered_global.csv"
  URLJP  = "https://dl.dropboxusercontent.com/s/6mztoeb6xf78g5w/COVID-19.csv"
  SAVE_NAMES = ["covid19_ww_confirmed_global.csv", "covid19_ww_deaths_global.csv","covid19_ww_recovered_global.csv", "covid19_jp.csv"]
  DIRCSV = "./static/csv/"
  COMP_FILES = ["base_covid19_ww_confirmed_global.csv",
    "base_covid19_ww_deaths_global.csv",
    "base_covid19_ww_recovered_global.csv",
    "covid19_jp.csv",
    "covid19_jp_all.csv",
    "covid19_jp_prf_cfm.csv",
    "covid19_jp_prf_dth.csv",
    "covid19_jp_prf_female.csv",
    "covid19_jp_prf_gen.csv",
    "covid19_jp_prf_male.csv",
    "covid19_jp_prf_rcv.csv",
    "covid19_ww_confirmed_global.csv",
    "covid19_ww_deaths_global.csv",
    "covid19_ww_recovered_global.csv",
    "daily_covid19_jp_prf_cfm.csv",
    "daily_covid19_jp_prf_dth.csv",
    "daily_covid19_jp_prf_female.csv",
    "daily_covid19_jp_prf_male.csv",
    "daily_covid19_jp_prf_rcv.csv",
    "daily_covid19_ww_confirmed_global.csv",
    "daily_covid19_ww_deaths_global.csv",
    "daily_covid19_ww_recovered_global.csv"]
  urls = [URLWW1, URLWW2, URLWW3, URLJP]
  del_cols = ["Province/State", "Lat", "Long"]

  are_allfiles = True
  for x in COMP_FILES:
    if os.path.isfile(DIRCSV + x) == False: are_allfiles = False
  if are_allfiles:
    today = datetime.date.today()
    dt = os.path.getmtime(DIRCSV + COMP_FILES[16])
    dt = datetime.datetime.fromtimestamp(dt)
    dt = dt.strftime('%Y-%m-%d')
    if today == dt: return
  
  ##############################################################################
  ## Reshape of worldwide COVID19 daily numbers csv
  ##############################################################################
  today = datetime.date.today()
  save_names = SAVE_NAMES.copy()
  del save_names[-1]
  i = 0
  for f_name in save_names:
    # Download a target file if it is not exist
    if os.path.isfile(DIRCSV + f_name) == False:
      r = requests.get(urls[i], headers=HEADERS_DIC)
      with open(DIRCSV + 'base_' + f_name, mode='w', encoding='utf_8_sig') as f:
        f.write(r.text)

      ## Reshape csv file
      df = pd.read_csv(DIRCSV + 'base_' + f_name, encoding='utf_8_sig')
      ### Drop unnessesary columns
      df = df.drop(del_cols, axis=1) 
      ### Reshape date style          
      cols = df.columns
      dates = []
      for col in cols:
        if col == 'Country/Region':
          dates.append(col)
        else:
          day = datetime.datetime.strptime(col, '%m/%d/%y')  
          dates.append(day.strftime('%Y-%m-%d'))
      df.columns = dates
      ### Group and sum each country
      df = df.groupby("Country/Region").sum()
      ### Save as csv file
      df.to_csv(DIRCSV + f_name, encoding='utf_8_sig')
      ### Delete dataframe instance
      del df
    else:
      ### Get file datetime
      dt = os.path.getmtime(DIRCSV + f_name)
      dt = datetime.datetime.fromtimestamp(dt)
      dt = dt.strftime('%Y-%m-%d')
      ## Download and reshape csv file when the exist file not made on today
      if str(dt) != str(today):
        r = requests.get(urls[i], headers=HEADERS_DIC)
        with open(DIRCSV + 'base_' + f_name, mode='w', encoding='utf_8_sig') as f:
          f.write(r.text)
        df = pd.read_csv(DIRCSV + 'base_' + f_name, encoding='utf_8_sig')
        df = df.drop(del_cols, axis=1) 
        cols = df.columns
        if cols[0] != 'Country/Region':
          df = df.reset_index()
        dates = []
        for col in cols:
          if col == 'Country/Region':
            dates.append(col)
          else:
            day = datetime.datetime.strptime(col, '%m/%d/%y')  
            dates.append(day.strftime('%Y-%m-%d'))
        df.columns = dates
        ### Group and sum each countory
        df = df.groupby("Country/Region").sum()
        df = df.set_index('Country/Region')
        ### Save as csv file
        df.to_csv(DIRCSV + f_name, encoding='utf_8_sig')

        ### Delete dataframe instance
        del df
    i += 1
    ### Daily csv
    df = pd.read_csv(DIRCSV + f_name, encoding='utf_8_sig')
    df = df.set_index('Country/Region')
    for j in range(0, len(df.index), 1):
        for k in range(len(df.columns)-1, 0, -1):
          df.iloc[j, k] = df.iloc[j, k] - df.iloc[j, k-1]
    df.to_csv(DIRCSV + 'daily_' + f_name, encoding='utf_8_sig')
    del df
    

  ##############################################################################
  ## Reshape of Japanese COVID19 daily numbers csv
  ##############################################################################
  f_name = SAVE_NAMES[3]
  today = datetime.date.today()
  # Download a target file if it is not exist
  if os.path.isfile(DIRCSV + f_name) == False:
    r = requests.get(urls[3], headers=HEADERS_DIC)
    with open(DIRCSV + f_name, mode='w', encoding='utf_8_sig') as f:
      f.write(r.text)
  else:
    dt = os.path.getmtime(DIRCSV + f_name)
    dt = datetime.datetime.fromtimestamp(dt)
    dt = dt.strftime('%Y-%m-%d')
    if today != dt:
      r = requests.get(urls[3], headers=HEADERS_DIC)
      with open(DIRCSV + f_name, mode='w', encoding='utf_8_sig') as f:
        f.write(r.text)
    
  df = pd.read_csv(DIRCSV + f_name, encoding='utf_8_sig',
                   dtype={1: str, 3: str, 4: str, 23: str, 24: str, 25: str, 31: str, 32: str, 42: str, 43: str})
  ## Reshape csv file
  
  ################################################################################
  ## Download of Japanese COVID19 data
  ## Reshape the data
  ################################################################################
  df = df.loc[:, ["通し", "受診都道府県", "確定日", "更新日時", "年代", "性別", "ステータス", "人数"]]
  df.columns = ["No.", "Prefecture", "Date", "Saved_Date", "Genelation", "Sex", "Status", "Number"]
  df['Status'] = df['Status'].replace(np.nan, "confirmed")
  df['Status'] = df['Status'].replace('退院', "recovered")
  df['Status'] = df['Status'].replace('死(.*)', "death", regex=True)
  df['Prefecture'] = df['Prefecture'].replace('中部国際空港', '愛知県')
  df['Prefecture'] = df['Prefecture'].replace('成田空港', '千葉県')
  df['Prefecture'] = df['Prefecture'].replace('羽田空港', '東京都')
  df['Prefecture'] = df['Prefecture'].replace('関西国際空港', '大阪府')
  sr = df['Saved_Date']
  modified_date = str(sr[0])
  modified_date = datetime.datetime.strptime(modified_date, '%m/%d/%Y %H:%M')
  modified_date = modified_date.strftime('%Y-%m-%d %H:%M')
  df.at[0, 'Saved_Date'] = modified_date
  df = df.dropna(subset=["Date"])
  dates = []
  for date in df['Date']:
    day = datetime.datetime.strptime(str(date), '%m/%d/%Y')
    dates.append(day.strftime('%Y-%m-%d'))
  df['Date'] = dates

  df.to_csv(DIRCSV + "covid19_jp_all.csv", encoding='utf_8_sig')

  ## 日本の都道府県別・日付別発症者数リスト
  dfc = df.drop(["No.", "Saved_Date", "Genelation", "Sex", "Status"], axis=1)
  dfc = dfc.groupby(["Prefecture", "Date"]).sum().unstack()
  dfc = dfc.replace(np.nan, 0.0)
  dfc.columns = get_converted_multi_columns(dfc, just_second=True)
  dfc.to_csv(DIRCSV + "daily_covid19_jp_prf_cfm.csv", encoding='utf_8_sig')
  # Daily
  for i in range(0, len(dfc.index), 1):
      for j in range(1, len(dfc.columns), 1):
        dfc.iloc[i, j] = int(dfc.iloc[i, j-1]) + int(dfc.iloc[i, j])
  dfc.to_csv(DIRCSV + "covid19_jp_prf_cfm.csv", encoding='utf_8_sig')

  ## 日本の都道府県別・日付別死者数リスト
  dfc = df.drop(["No.", "Saved_Date", "Genelation", "Sex", "Number"], axis=1)
  dfc = dfc[dfc["Status"] != "confirmed"]
  dfc = dfc[dfc["Status"] != "recovered"]
  dfc["Status"] = 1
  dfc = dfc.groupby(["Prefecture", "Date"]).sum().unstack()
  dfc = dfc.replace(np.nan, 0.0)
  dfc.columns = get_converted_multi_columns(dfc, just_second=True)
  dfc.to_csv(DIRCSV + "daily_covid19_jp_prf_dth.csv", encoding='utf_8_sig')
  # Daily
  for i in range(0, len(dfc.index), 1):
      for j in range(1, len(dfc.columns), 1):
        dfc.iloc[i, j] = int(dfc.iloc[i, j-1]) + int(dfc.iloc[i, j])
  dfc.to_csv(DIRCSV + "covid19_jp_prf_dth.csv", encoding='utf_8_sig')

  ## 日本の都道府県別・日付別退院者数リスト
  dfc = df.drop(["No.", "Saved_Date", "Genelation", "Sex", "Number"], axis=1)
  dfc = dfc[dfc["Status"] != "confirmed"]
  dfc = dfc[dfc["Status"] != "death"]
  dfc["Status"] = 1
  dfc = dfc.groupby(["Prefecture", "Date"]).sum().unstack()
  dfc = dfc.replace(np.nan, 0.0)
  dfc.columns = get_converted_multi_columns(dfc, just_second=True)
  dfc.to_csv(DIRCSV + "daily_covid19_jp_prf_rcv.csv", encoding='utf_8_sig')
  # Daily
  for i in range(0, len(dfc.index), 1):
      for j in range(1, len(dfc.columns), 1):
        dfc.iloc[i, j] = int(dfc.iloc[i, j-1]) + int(dfc.iloc[i, j])
  dfc.to_csv(DIRCSV + "covid19_jp_prf_rcv.csv", encoding='utf_8_sig')

  ## 日本の都道府県別・ 男性数リスト
  dfc = df.drop(["No.", "Saved_Date", "Genelation", "Status", "Number"], axis=1)
  dfc = dfc[dfc["Sex"] != "女性"]
  dfc["Sex"] = 1
  dfc = dfc.groupby(["Prefecture", "Date"]).sum().unstack()
  dfc = dfc.replace(np.nan, 0.0)
  dfc.columns = get_converted_multi_columns(dfc, just_second=True)
  dfc.to_csv(DIRCSV + "daily_covid19_jp_prf_male.csv", encoding='utf_8_sig')
  # Daily
  for i in range(0, len(dfc.index), 1):
      for j in range(1, len(dfc.columns), 1):
        dfc.iloc[i, j] = int(dfc.iloc[i, j-1]) + int(dfc.iloc[i, j])
  dfc.to_csv(DIRCSV + "covid19_jp_prf_male.csv", encoding='utf_8_sig')

  ## 日本の都道府県別・ 女性数リスト
  dfc = df.drop(["No.", "Saved_Date", "Genelation", "Status", "Number"], axis=1)
  dfc = dfc[dfc["Sex"] != "男性"]
  dfc["Sex"] = 1
  dfc = dfc.groupby(["Prefecture", "Date"]).sum().unstack()
  dfc = dfc.replace(np.nan, 0.0)
  dfc.columns = get_converted_multi_columns(dfc, just_second=True)
  dfc.to_csv(DIRCSV + "daily_covid19_jp_prf_female.csv", encoding='utf_8_sig')
  # Daily
  for i in range(0, len(dfc.index), 1):
      for j in range(1, len(dfc.columns), 1):
        dfc.iloc[i, j] = int(dfc.iloc[i, j-1]) + int(dfc.iloc[i, j])
  dfc.to_csv(DIRCSV + "covid19_jp_prf_female.csv", encoding='utf_8_sig')

  ## 日本の都道府県別・ 年代別リスト
  dfc = df.drop(["No.", "Date", "Saved_Date", "Sex", "Status"], axis=1)
  dfc = dfc.replace(np.nan, 99.0)
  dfc = dfc.replace("0-10", 1.0)
  dfc = dfc.replace("10", 10.0)
  dfc = dfc.replace("20", 20.0)
  dfc = dfc.replace("30", 30.0)
  dfc = dfc.replace("40", 40.0)
  dfc = dfc.replace("50", 50.0)
  dfc = dfc.replace("60", 60.0)
  dfc = dfc.replace("70", 70.0)
  dfc = dfc.replace("80", 80.0)
  dfc = dfc.replace("90", 90.0)
  dfc = dfc.replace(10, 10.0)
  dfc = dfc.replace(20, 20.0)
  dfc = dfc.replace(30, 30.0)
  dfc = dfc.replace(40, 40.0)
  dfc = dfc.replace(50, 50.0)
  dfc = dfc.replace(60, 60.0)
  dfc = dfc.replace(70, 70.0)
  dfc = dfc.replace(80, 80.0)
  dfc = dfc.replace(90, 90.0)
  dfc["Genelation"] = dfc["Genelation"].replace("不明", 99.0)
  dfc["Genelation"] = dfc["Genelation"].astype(int)
  dfc = dfc.groupby(["Prefecture", "Genelation"]).sum().unstack()
  dfc.columns = get_converted_multi_columns(dfc, just_second=True)
  dfc = dfc.replace(np.nan, 0.0)
  dfc = dfc.astype(int)
  dfc.to_csv(DIRCSV + "covid19_jp_prf_gen.csv", encoding='utf_8_sig')

if __name__ == "__main__":
  down_load()
"""

  
with open('dldata.py', mode='w', encoding='utf_8_sig') as f:
  f.write(dldata)

### Make files to use dashboard

In [0]:
!pip install plotly
!pip install dash
!pip install dash-html-components
!pip install dash-core-components
!pip install dash-table
!pip install dash-daq 
!pip install dash_bootstrap_components
!pip install gunicorn

## Dashboard

### Build application server

### app.pyの作成

In [0]:
dashfile = """
# -*- coding: utf-8 -*-
import os
import datetime
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import numpy as np
import pandas as pd
import dldata

dldata.down_load()


DIRCSV = 'static/csv/'
CSV_FILES = ('covid19_ww_confirmed_global.csv',
            'covid19_ww_deaths_global.csv',
            'covid19_ww_recovered_global.csv',
            'covid19_jp_prf_cfm.csv',
            'covid19_jp_prf_dth.csv',
            'covid19_jp_prf_rcv.csv',
            'covid19_jp_prf_male.csv',
            'covid19_jp_prf_female.csv',
            'covid19_jp_prf_gen.csv')

df00 = pd.read_csv(DIRCSV + CSV_FILES[0], header=0, index_col=0)
df01 = pd.read_csv(DIRCSV + CSV_FILES[1], header=0, index_col=0)
df02 = pd.read_csv(DIRCSV + CSV_FILES[2], header=0, index_col=0)
df03 = pd.read_csv(DIRCSV + CSV_FILES[3], header=0, index_col=0)
df04 = pd.read_csv(DIRCSV + CSV_FILES[4], header=0, index_col=0)
df05 = pd.read_csv(DIRCSV + CSV_FILES[5], header=0, index_col=0)
df06 = pd.read_csv(DIRCSV + CSV_FILES[6], header=0, index_col=0)
df07 = pd.read_csv(DIRCSV + CSV_FILES[7], header=0, index_col=0)
df08 = pd.read_csv(DIRCSV + CSV_FILES[8], header=0, index_col=0)


#df00 = df00.set_index('Country/Region')
df00.loc['Worldwide'] = df00.sum()
df00_x = df00.columns.tolist()
df00_y = df00.loc['Worldwide', ].tolist()
df00_z = df00.index.tolist()
today_df00 = df00_x[-1]
today_number_df00 = df00_y[-1]
dict00 = []
for x in df00_z:
  dict00.append({'label': x, 'value': x})

df01.loc['Worldwide'] = df01.sum()
df01_x = df01.columns.tolist()
df01_y = df01.loc['Worldwide', ].tolist()
df01_z = df01.index.tolist()
today_df01 = df01_x[-1]
today_number_df01 = df01_y[-1]
dict01 = []
for x in df01_z:
  dict01.append({'label': x, 'value': x})

df02.loc['Worldwide'] = df02.sum()
df02_x = df02.columns.tolist()
df02_y = df02.loc['Worldwide', ].tolist()
df02_z = df02.index.tolist()
today_df02 = df02_x[-1]
today_number_df02 = df02_y[-1]
dict02 = []
for x in df02_z:
  dict02.append({'label': x, 'value': x})

df03.loc['全日本'] = df03.sum()
df03_x = df03.columns.tolist()
df03_y = df03.loc['全日本', ].tolist()
df03_z = df03.index.tolist()
today_df03 = df03_x[-1]
today_number_df03 = df03_y[-1]
dict03 = []
for x in df03_z:
  dict03.append({'label': x, 'value': x})

df04.loc['全日本'] = df04.sum()
df04_x = df04.columns.tolist()
df04_y = df04.loc['全日本', ].tolist()
df04_z = df04.index.tolist()
today_df04 = df04_x[-1]
today_number_df04 = df04_y[-1]
dict04 = []
for x in df04_z:
  dict04.append({'label': x, 'value': x})

df05.loc['全日本'] = df05.sum()
df05_x = df05.columns.tolist()
df05_y = df05.loc['全日本', ].tolist()
df05_z = df05.index.tolist()
today_df05 = df05_x[-1]
today_number_df05 = df05_y[-1]
dict05 = []
for x in df05_z:
  dict05.append({'label': x, 'value': x})

df06.loc['全日本'] = df06.sum()
df06_x = df06.columns.tolist()
df06_y = df06.loc['全日本', ].tolist()
df06_z = df06.index.tolist()
today_df06 = df06_x[-1]
today_number_df06 = df06_y[-1]
dict06 = []
for x in df06_z:
  dict06.append({'label': x, 'value': x})

df07.loc['全日本'] = df07.sum()
df07_x = df07.columns.tolist()
df07_y = df07.loc['全日本', ].tolist()
df07_z = df07.index.tolist()
today_df07 = df07_x[-1]
today_number_df07 = df07_y[-1]
dict07 = []
for x in df07_z:
  dict07.append({'label': x, 'value': x})

df08.loc['全日本'] = df08.sum()
df08_x = df08.columns.tolist()
df08_y = df08.loc['全日本', ].tolist()
df08_z = df08.index.tolist()
today_df08 = df08_x[-1]
today_number_df08 = df08_y[-1]
dict08 = []
for x in df08_z:
  dict08.append({'label': x, 'value': x})

txt00 = str(today_df00) + '全世界COVID-19感染者総数：' + str(today_number_df00) + '名'
txt01 = str(today_df01) + '全世界COVID-19死者総数　：' + str(today_number_df01) + '名'
txt02 = str(today_df02) + '全世界COVID-19快復者総数：' + str(today_number_df02) + '名'

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
server = app.server

# 背景色と文字色の設定
colors = {
    'background': '#111111',
    'text': '#7FDBFF',
    'text01': '#111111'
}

app.layout = html.Div(style={'backgroundColor': colors['background']}, children=[
  html.H1(
    children='COVID19ダッシュボード',
    style={'textAlign': 'center',
          'color': colors['text']
    }
  ),

  html.H6(
    children='COVID19ワールドワイド及び日本国内感染者数情報', 
    style={'textAlign': 'center',
      'color': colors['text']
    }
  ),
  html.H5(
    children = txt00,
    style={'color': colors['text']}
  ),
  html.H5(
    children = txt01,
    style={'color': colors['text']}
  ),
  html.H5(
    children = txt02,
    style={'color': colors['text']}
  ),
##
# Plot WW confirmed
##
  html.Div([
      dcc.Input(
        id = 'textarea00',
        value = 'COVID19感染者数情報',
        style={'textAlign': 'center',
          'color': colors['text01'],
          'bg_color': colors['background'],
          'width': '100%'
          }
      ),

      dcc.Dropdown(
        id='my_ticker_symbol00',
        options=dict00,
        value='Worldwide',
      ),

      dcc.Graph(
          id='example-graph00',
          figure={
              'data':[
                {'x': df00_x, 'y': df00_y}
              ],
              'layout': {
                  'plot_bgcolor': colors['background'],
                  'paper_bgcolor': colors['background'],
                  'font': {
                      'color': colors['text']
                  },
              }
          }
      ),
    ],style={'width': '32%', 'display': 'inline-block'}),
##
# Plot WW deaths
##
    html.Div([
      dcc.Input(
        id = 'textarea01',
        value = 'COVID19死者数情報',
        style={'textAlign': 'center',
          'color': colors['text01'],
          'width': '100%'}
      ),

      dcc.Dropdown(
        id='my_ticker_symbol01',
        options=dict01,
        value='Worldwide',
      ),

      dcc.Graph(
          id='example-graph01',
          figure={
              'data':[
                {'x': df01_x, 'y': df01_y}
              ],
              'layout': {
                  'plot_bgcolor': colors['background'],
                  'paper_bgcolor': colors['background'],
                  'font': {
                      'color': colors['text']
                  }
              }
          }
      )
    ],style={'width': '32%', 'display': 'inline-block'}),
##
# Plot WW recovered
##
    html.Div([
      dcc.Input(
        id = 'textarea02',
        value = 'COVID19退院者数情報',
        style={'textAlign': 'center',
          'color': colors['text01'],
          'width': '100%'}
      ),

      dcc.Dropdown(
        id='my_ticker_symbol02',
        options=dict02,
        value='Worldwide',
      ),

      dcc.Graph(
          id='example-graph02',
          figure={
              'data':[
                {'x': df02_x, 'y': df02_y}
              ],
              'layout': {
                  'plot_bgcolor': colors['background'],
                  'paper_bgcolor': colors['background'],
                  'font': {
                      'color': colors['text']
                  }
              }
          }
      )
    ],style={'width': '32%', 'display': 'inline-block'}),
##
# Plot Japan Confirmed
##
    html.Div([
      dcc.Input(
        id = 'textarea03',
        value = 'COVID19感染者数情報',
        style={'textAlign': 'center',
          'color': colors['text01'],
          'width': '100%'}
      ),

      dcc.Dropdown(
        id='my_ticker_symbol03',
        options=dict03,
        value='全日本',
      ),

      dcc.Graph(
          id='example-graph03',
          figure={
              'data':[
                {'x': df03_x, 'y': df03_y}
              ],
              'layout': {
                  'plot_bgcolor': colors['background'],
                  'paper_bgcolor': colors['background'],
                  'font': {
                      'color': colors['text']
                  }
              }
          }
      )
    ],style={'width': '32%', 'display': 'inline-block'}),
##
# Plot Japan deaths
##
    html.Div([
      dcc.Input(
        id = 'textarea04',
        value = 'COVID19死者数情報',
        style={'textAlign': 'center',
          'color': colors['text01'],
          'width': '100%'}
      ),

      dcc.Dropdown(
        id='my_ticker_symbol04',
        options=dict04,
        value='全日本',
      ),

      dcc.Graph(
          id='example-graph04',
          figure={
              'data':[
                {'x': df04_x, 'y': df04_y}
              ],
              'layout': {
                  'plot_bgcolor': colors['background'],
                  'paper_bgcolor': colors['background'],
                  'font': {
                      'color': colors['text']
                  }
              }
          }
      )
    ],style={'width': '32%', 'display': 'inline-block'}),
##
# Plot Japan recovered
##
    html.Div([
      dcc.Input(
        id = 'textarea05',
        value = 'COVID19退院者数情報',
        style={'textAlign': 'center',
          'color': colors['text01'],
          'width': '100%'}
      ),

      dcc.Dropdown(
        id='my_ticker_symbol05',
        options=dict05,
        value='全日本',
      ),

      dcc.Graph(
          id='example-graph05',
          figure={
              'data':[
                {'x': df05_x, 'y': df05_y}
              ],
              'layout': {
                  'plot_bgcolor': colors['background'],
                  'paper_bgcolor': colors['background'],
                  'font': {
                      'color': colors['text']
                  }
              }
          }
      )
    ],style={'width': '32%', 'display': 'inline-block'}),
##
# Plot Japan male
##
    html.Div([
      dcc.Input(
        id = 'textarea06',
        value = 'COVID19男性感染者数情報',
        style={'textAlign': 'center',
          'color': colors['text01'],
          'width': '100%'}
      ),

      dcc.Dropdown(
        id='my_ticker_symbol06',
        options=dict06,
        value='全日本',
      ),

      dcc.Graph(
          id='example-graph06',
          figure={
              'data':[
                {'x': df06_x, 'y': df06_y}
              ],
              'layout': {
                  'plot_bgcolor': colors['background'],
                  'paper_bgcolor': colors['background'],
                  'font': {
                      'color': colors['text']
                  }
              }
          }
      )
    ],style={'width': '32%', 'display': 'inline-block'}),
##
# Plot Japan female
##
    html.Div([
      dcc.Input(
        id = 'textarea07',
        value = 'COVID19女性感染者数情報',
        style={'textAlign': 'center',
          'color': colors['text01'],
          'width': '100%'}
      ),

      dcc.Dropdown(
        id='my_ticker_symbol07',
        options=dict07,
        value='全日本',
      ),

      dcc.Graph(
          id='example-graph07',
          figure={
              'data':[
                {'x': df07_x, 'y': df07_y}
              ],
              'layout': {
                  'plot_bgcolor': colors['background'],
                  'paper_bgcolor': colors['background'],
                  'font': {
                      'color': colors['text']
                  }
              }
          }
      )
    ],style={'width': '32%', 'display': 'inline-block'}),
##
# Plot Japan generation
##
    html.Div([
      dcc.Input(
        id = 'textarea08',
        value = 'COVID19年代別感染者数情報',
        style={'textAlign': 'center',
          'color': colors['text01'],
          'width': '100%'}
      ),

      dcc.Dropdown(
        id='my_ticker_symbol08',
        options=dict08,
        value='全日本',
      ),

      dcc.Graph(
          id='example-graph08',
          figure={
              'data':[
                {'x': df08_x, 'y': df08_y}
              ],
              'layout': {
                  'plot_bgcolor': colors['background'],
                  'paper_bgcolor': colors['background'],
                  'font': {
                      'color': colors['text']
                  }
              }
          }
      )
    ],style={'width': '32%', 'display': 'inline-block'}),

])

###############################################################################
##
# Callback WW confirmed
##
@app.callback(
  Output('textarea00', 'value'),
  [Input('my_ticker_symbol00', 'value')]
)

def update_text(value):
  value = 'COVID-19感染者数情報:  ' + value
  return value

@app.callback(
  Output('example-graph00', 'figure'),
  [Input('my_ticker_symbol00', 'value')])

def update_graph(stock_ticker):
  df00_y = df00.loc[stock_ticker, ].tolist()
  fig = {
    'data':[
      {'x': df00_x, 'y': df00_y}
    ],
    'layout': {
      'title': stock_ticker,
      'plot_bgcolor': colors['background'],
      'paper_bgcolor': colors['background'],
      'font': {'color': colors['text']}
    }
  }
  return fig

##
# Callback WW deaths
##
@app.callback(
  Output('textarea01', 'value'),
  [Input('my_ticker_symbol01', 'value')]
)

def update_text(value):
  value = 'COVID-19死者数情報:  ' + value
  return value

@app.callback(
  Output('example-graph01', 'figure'),
  [Input('my_ticker_symbol01', 'value')])

def update_graph(stock_ticker):
  df01_y = df01.loc[stock_ticker, ].tolist()
  fig = {
    'data':[
      {'x': df01_x, 'y': df01_y}
    ],
    'layout': {
      'title': stock_ticker,
      'plot_bgcolor': colors['background'],
      'paper_bgcolor': colors['background'],
      'font': {'color': colors['text']}
    }
  }
  return fig

##
# Callback WW recovered
##
@app.callback(
  Output('textarea02', 'value'),
  [Input('my_ticker_symbol02', 'value')]
)

def update_text(text):
  value = str('COVID-19快復者数情報:  ' + text)
  return value

@app.callback(
  Output('example-graph02', 'figure'),
  [Input('my_ticker_symbol02', 'value')])

def update_graph(stock_ticker):
  df02_y = df02.loc[stock_ticker, ].tolist()
  fig = {
    'data':[
      {'x': df02_x, 'y': df02_y}
    ],
    'layout': {
      'title': stock_ticker,
      'plot_bgcolor': colors['background'],
      'paper_bgcolor': colors['background'],
      'font': {'color': colors['text']}
    }
  }
  return fig

##
# Callback JP confirmed
##
@app.callback(
  Output('textarea03', 'value'),
  [Input('my_ticker_symbol03', 'value')]
)

def update_text(text):
  value = str('COVID-19感染者数情報:  ' + text)
  return value

@app.callback(
  Output('example-graph03', 'figure'),
  [Input('my_ticker_symbol03', 'value')])

def update_graph(stock_ticker):
  df03_y = df03.loc[stock_ticker, ].tolist()
  fig = {
    'data':[
      {'x': df03_x, 'y': df03_y}
    ],
    'layout': {
      'title': stock_ticker,
      'plot_bgcolor': colors['background'],
      'paper_bgcolor': colors['background'],
      'font': {'color': colors['text']}
    }
  }
  return fig

##
# Callback JP deaths
##
@app.callback(
  Output('textarea04', 'value'),
  [Input('my_ticker_symbol04', 'value')]
)

def update_text(text):
  value = str('COVID-19死者数情報:  ' + text)
  return value

@app.callback(
  Output('example-graph04', 'figure'),
  [Input('my_ticker_symbol04', 'value')])

def update_graph(stock_ticker):
  df04_y = df04.loc[stock_ticker, ].tolist()
  fig = {
    'data':[
      {'x': df04_x, 'y': df04_y}
    ],
    'layout': {
      'title': stock_ticker,
      'plot_bgcolor': colors['background'],
      'paper_bgcolor': colors['background'],
      'font': {'color': colors['text']}
    }
  }
  return fig

##
# Callback JP recovered
##
@app.callback(
  Output('textarea05', 'value'),
  [Input('my_ticker_symbol05', 'value')]
)

def update_text(text):
  value = str('COVID-19快復者数情報:  ' + text)
  return value

@app.callback(
  Output('example-graph05', 'figure'),
  [Input('my_ticker_symbol05', 'value')])

def update_graph(stock_ticker):
  df05_y = df05.loc[stock_ticker, ].tolist()
  fig = {
    'data':[
      {'x': df05_x, 'y': df05_y}
    ],
    'layout': {
      'title': stock_ticker,
      'plot_bgcolor': colors['background'],
      'paper_bgcolor': colors['background'],
      'font': {'color': colors['text']}
    }
  }
  return fig

##
# Callback JP male
##
@app.callback(
  Output('textarea06', 'value'),
  [Input('my_ticker_symbol06', 'value')]
)

def update_text(text):
  value = str('COVID-19男性感染者数情報:  ' + text)
  return value

@app.callback(
  Output('example-graph06', 'figure'),
  [Input('my_ticker_symbol06', 'value')])

def update_graph(stock_ticker):
  df06_y = df06.loc[stock_ticker, ].tolist()
  fig = {
    'data':[
      {'x': df06_x, 'y': df06_y}
    ],
    'layout': {
      'title': stock_ticker,
      'plot_bgcolor': colors['background'],
      'paper_bgcolor': colors['background'],
      'font': {'color': colors['text']}
    }
  }
  return fig

##
# Callback JP female
##
@app.callback(
  Output('textarea07', 'value'),
  [Input('my_ticker_symbol07', 'value')]
)

def update_text(text):
  value = str('COVID-19女性感染者数情報:  ' + text)
  return value

@app.callback(
  Output('example-graph07', 'figure'),
  [Input('my_ticker_symbol07', 'value')])

def update_graph(stock_ticker):
  df07_y = df07.loc[stock_ticker, ].tolist()
  fig = {
    'data':[
      {'x': df07_x, 'y': df07_y}
    ],
    'layout': {
      'title': stock_ticker,
      'plot_bgcolor': colors['background'],
      'paper_bgcolor': colors['background'],
      'font': {'color': colors['text']}
    }
  }
  return fig

##
# Callback JP generation
##
@app.callback(
  Output('textarea08', 'value'),
  [Input('my_ticker_symbol08', 'value')]
)

def update_text(text):
  value = str('COVID-19年齢別感染者数情報:  ' + text)
  return value

@app.callback(
  Output('example-graph08', 'figure'),
  [Input('my_ticker_symbol08', 'value')])

def update_graph(stock_ticker):
  df08_y = df08.loc[stock_ticker, ].tolist()
  fig = {
    'data':[
      {'x': df08_x, 'y': df08_y}
    ],
    'layout': {
      'title': stock_ticker,
      'plot_bgcolor': colors['background'],
      'paper_bgcolor': colors['background'],
      'font': {'color': colors['text']}
    }
  }
  return fig

#if __name__ == '__main__':
  #app.run_server(debug=True)
"""

with open('app.py', mode='w', encoding='utf-8') as f:
  f.write(dashfile)

### ngrok tonneling through Colaboratory

In [0]:
get_ipython().system_raw('./ngrok http 8000 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
  "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

#!python app.py
! gunicorn app:server


### Nginx

In [0]:
!wget https://nginx.org/download/nginx-1.18.0.tar.gz
!tar -zxvf nginx-1.18.0.tar.gz

In [0]:
# lsofコマンドは「LiSt Open Files」（開いているファイル群を列挙する）という言葉に由来します。その名の通り、「プロセスが開いているファイル」を表示するコマンドです。
!sudo lsof -i:80


In [0]:
!cd nginx-1.18.0
!sudo apt-get install nginx

In [0]:
!mkdir config

In [0]:
guni_set = """
import os

# Socket Definition
socket_path = 'unix:/tmp/gunicorn.sock'
bind = socket_path
# bind = '127.0.0.1:' + str(os.getenv('PORT', 9876))
proc_name = 'Infrastructure-Practice-Flask'
workers = 1
"""

with open('./config/gunicorn_settings.py', mode='w', encoding='utf_8_sig') as f:
  f.write(guni_set)

        listen        80;                          # 80番ポートで接続を待つ
        server_name   MY_SERVER;                   # サーバ名はMY_SERVER
        charset       UTF-8;                       # レスポンスヘッダのContent-typeをUTF-8に指定
        error_page    404  /404_not_found.html;    # ステータスコードが404のときに404_not_found.htmlへ内部的にリダイレクト

        # /indexへのアクセスを
        location /index {
            proxy_pass          http://127.0.0.1:9876;  # 127.0.0.1の9876ポートへリバースプロキシする
            proxy_set_header    Host    $host;          # プロキシされたサーバへ渡すリクエストヘッダのうち、Hostに$hostを設定し直す
　　　　　　proxy_read_timeout  60s;                    # プロキシされたサーバからの応答が60秒なかったらタイムアウトとする

In [0]:
nginxconf = """
worker_processes  1;

events {
    worker_connections 512;
}

http {
    upstream app_server{
        server unix:/tmp/gunicorn.sock;
    }
    server {
        listen        80;
        server_name   appserver;
        charset       UTF-8;
        error_page    404  /404_not_found.html;
        location = /favicon.ico { access_log off; log_not_found off;}

        location /static/ {
            alias /usr/share/nginx/html/static;
        }
    }
}
"""

with open("./config/nginx.conf", mode='w', encoding='utf_8_sig') as f:
  f.write(nginxconf)

Webサーバの起動
アプリケーションサーバの起動

In [0]:
!nginx -c /content/config/nginx.conf

!gunicorn app:server -c /content/config/gunicorn_settings.py

In [0]:
!nginx -s stop

In [0]:
!sudo systemctl start nginx
!sudo systemctl status nginx

In [0]:
! gunicorn app:server

In [0]:
!sudo lsof -i | grep nginx

In [0]:
!sudo kill -9 3675
!sudo kill -9 3676

### Configuration files

In [0]:
rt = """
Python 3.6.9
"""

with open('runtime.txt', mode='w', encoding='utf_8_sig') as f:
  f.write(rt)

#herokuではruntime.txtでpythonのバージョン指定等runtimeの条件設定ができるがPython3.6.9はサポートされていない

In [0]:
procf = """
web: bin/start-nginx bundle exec gunicorn --config config/gunicorn.conf.py --app app:server
"""

with open('procfile', mode='w', encoding='utf_8_sig') as f:
  f.write(procf)

In [0]:
require = """
numpy
pandas
requests
requests-html
seaborn
flask
plotly
dash
dash-html-components
dash-core-components
dash-table
dash-daq 
dash_bootstrap_components
gunicorn
"""

with open('requirements.txt', mode='w', encoding='utf_8_sig') as f:
  f.write(require)

In [0]:
#config/nginx.confを作成する

conf = """
worker_processes <%= ENV['NGINX_WORKERS'] || 4 %>;

events {
  use epoll;
  accept_mutex on;
  worker_connections 1024;
}

http {
  gzip on;
  gzip_comp_level 3;
  gzip_min_length 150;
  gzip_proxied any;
  gzip_types text/plain text/css text/json text/javascript
    application/javascript application/x-javascript application/json
    application/rss+xml application/vnd.ms-fontobject application/x-font-ttf
    application/xml font/opentype image/svg+xml text/xml;

  server_tokens off;

  log_format l2met 'measure#nginx.service=$request_time request_id=$http_x_request_id';
  access_log logs/nginx/access.log l2met;
  error_log logs/nginx/error.log;

  include mime.types;
  default_type application/octet-stream;
  sendfile on;

  # Must read the body in 5 seconds.
  client_body_timeout 5;

  upstream app_server {
    server unix:/tmp/nginx.socket fail_timeout=0;
  }

  server {
    listen <%= ENV["PORT"] %>;
    server_name _;
    keepalive_timeout 5;

    root /app/public; # path to your app

"""

with open('nginx.conf', mode='w', encoding='utf_8_sig') as f:
  f.write(conf)

### Directory listing for /

下記は静的なwebページの表示はできるが、Dashのような動的なページを表示するのには十分なリソースが無い模様。
!python -m http.server 8000（で、このアプリのフォルダー表示ができる）の代わりに! gunicorn app:serverで表示はできるが、ページ上で動的に表示を変える処理にはリソースが足りない模様。

In [0]:
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(8000)"))
!python -m http.server 8000

In [0]:
from IPython.display import Javascript

def show_port(port, height=400):
  display(Javascript("""
  (async ()=>{
    fm = document.createElement('iframe')
    fm.src = await google.colab.kernel.proxyPort(%s)
    fm.width = '95%%'
    fm.height = '%d'
    fm.frameBorder = 0
    document.body.append(fm)
  })();
  """ % (port, height) ))
get_ipython().system_raw('python3 -m http.server 8888 &') 
show_port(8888)

In [0]:
# To stop the server, you can call ps and kill the process.
!ps